### База данных "Учебная аналитика по курсу"

В разделе 3.5 в связи с громоздкостью таблиц публикую только решения.


## 3.5.2 Задание

Отобрать все шаги, в которых рассматриваются вложенные запросы (то есть в названии шага упоминаются вложенные запросы). Указать к какому уроку и модулю они относятся. Для этого вывести 3 поля:

* в поле **Модуль** указать номер модуля и его название через пробел;
* в поле **Урок** указать номер модуля, порядковый номер урока (**lesson_position**) через точку и название урока через пробел;
* в поле **Шаг** указать номер модуля, порядковый номер урока (**lesson_position**) через точку, порядковый номер шага (**step_position**) через точку и название шага через пробел.  
  
Длину полей **Модуль** и **Урок** ограничить 19 символами, при этом слишком длинные надписи обозначить многоточием в конце (16 символов - это номер модуля или урока, пробел и  название **Урока** или **Модуля** к ним присоединить "**...**"). Информацию отсортировать по возрастанию номеров модулей, порядковых номеров уроков и порядковых номеров шагов.

```mysql
SELECT
    CONCAT(LEFT(CONCAT(module_id, ' ', module_name), 16), '...') AS Модуль,
    CONCAT(LEFT(CONCAT(module_id, '.', lesson_position, ' ', lesson_name), 16), '...') AS Урок,
    CONCAT(module_id, '.', lesson_position, '.', step_position, ' ', step_name) AS Шаг
FROM
    module
    JOIN lesson USING(module_id)
    JOIN step USING(lesson_id)
WHERE
    step_name LIKE '%вложен%'
ORDER BY
    Модуль,
    Урок,
    Шаг;
```

## 3.5.3 Задание

Заполнить таблицу **step_keyword** следующим образом: если ключевое слово есть в названии шага, то включить в **step_keyword** строку с **id** шага и **id** ключевого слова. 

```mysql
INSERT INTO
    step_keyword(step_id, keyword_id)
SELECT
    step_id,
    keyword_id
FROM
    keyword, step
WHERE
    REGEXP_INSTR(step_name, CONCAT('\\b', keyword_name, '\\b')) > 0;

SELECT * FROM step_keyword;
```

## 3.5.4 Задание

Реализовать поиск по ключевым словам. Вывести шаги, с которыми связаны ключевые слова **MAX** и **AVG** одновременно. Для шагов указать **id** модуля, позицию урока в модуле, позицию шага в уроке через точку, после позиции шага перед заголовком - пробел. Позицию шага в уроке вывести в виде двух цифр (если позиция шага меньше 10, то перед цифрой поставить 0). Столбец назвать **Шаг**. Информацию отсортировать по первому столбцу в алфавитном порядке.

```mysql
SELECT
    CONCAT(module_id, '.', lesson_position, '.', LPAD(step_position, 2, '0'), ' ', step_name) AS Шаг
FROM
    module
    JOIN lesson USING(module_id)
    JOIN step USING(lesson_id)
    JOIN step_keyword USING(step_id)
    JOIN keyword USING(keyword_id)
WHERE
    keyword_name IN ('MAX', 'AVG')
GROUP BY
    Шаг
HAVING
    COUNT(*) = 2
ORDER BY
    1;
```

## 3.5.5 Выборка данных по нескольким условиям, оператор CASE. Задание

Посчитать, сколько студентов относится к каждой группе. Столбцы назвать **Группа, Интервал, Количество**. Указать границы интервала.

```mysql
SELECT
    Группа,
    CASE
        WHEN Группа = "I" THEN "от 0 до 10"
        WHEN Группа = "II" THEN "от 11 до 15"
        WHEN Группа = "III" THEN "от 16 до 27"
        ELSE "больше 27"
    END AS Интервал,
    COUNT(*) AS Количество
FROM
    (
    SELECT student_name, rate, 
    CASE
        WHEN rate <= 10 THEN "I"
        WHEN rate <= 15 THEN "II"
        WHEN rate <= 27 THEN "III"
        ELSE "IV"
    END AS Группа
    FROM      
        (
         SELECT student_name, count(*) as rate
         FROM 
             (
              SELECT student_name, step_id
              FROM 
                  student 
                  INNER JOIN step_student USING(student_id)
              WHERE result = "correct"
              GROUP BY student_name, step_id
             ) query_in
         GROUP BY student_name 
         ORDER BY 2
        ) query_in_1) query_in_2
GROUP BY
    Группа;
```

## 3.5.6 Табличные выражения, оператор WITH. Задание

Исправить запрос примера так: для шагов, которые  не имеют неверных ответов,  указать 100 как процент успешных попыток, если же шаг не имеет верных ответов, указать 0. Информацию отсортировать сначала по возрастанию успешности, а затем по названию шага в алфавитном порядке.

```mysql
WITH get_count_correct (st_n_c, count_correct) 
  AS (
    SELECT step_name, count(*)
    FROM 
        step 
        INNER JOIN step_student USING (step_id)
    WHERE result = "correct"
    GROUP BY step_name
   ),
  get_count_wrong (st_n_w, count_wrong) 
  AS (
    SELECT step_name, count(*)
    FROM 
        step 
        INNER JOIN step_student USING (step_id)
    WHERE result = "wrong"
    GROUP BY step_name
   )  
SELECT st_n_c AS Шаг,
    IFNULL(ROUND(count_correct / (count_correct + count_wrong) * 100), 100) AS Успешность
FROM  
    get_count_correct 
    LEFT JOIN get_count_wrong ON st_n_c = st_n_w
UNION
SELECT st_n_w AS Шаг,
    IFNULL(ROUND(count_correct / (count_correct + count_wrong) * 100), 0) AS Успешность
FROM  
    get_count_correct 
    RIGHT JOIN get_count_wrong ON st_n_c = st_n_w
ORDER BY 2, 1;
```

## 3.5.7 Задание

Вычислить прогресс пользователей по курсу. Прогресс вычисляется как отношение верно пройденных шагов к общему количеству шагов в процентах, округленное до целого. В нашей базе данные о решениях занесены не для всех шагов, поэтому общее количество шагов определить как количество различных шагов в таблице **step_student**.

Тем пользователям, которые прошли все шаги (прогресс = 100%) выдать "Сертификат с отличием". Тем, у кого прогресс больше или равен 80% - "Сертификат". Для остальных записей в столбце **Результат** задать пустую строку ("").

Информацию отсортировать по убыванию прогресса, затем по имени пользователя в алфавитном порядке.

```mysql
SET @max_progress = (SELECT COUNT(DISTINCT step_id) FROM step_student);
    
WITH student_progress (student_id, progress) AS
    (
     SELECT
        student_id,
        ROUND(COUNT(DISTINCT step_id) / @max_progress * 100) AS progress
     FROM
         step_student
     WHERE
         result = "correct"
     GROUP BY
         student_id
    )
SELECT
    student_name AS Студент,
    progress AS Прогресс,
    CASE
        WHEN progress = 100 THEN "Сертификат с отличием"
        WHEN progress BETWEEN 80 AND 99 THEN "Сертификат"
        ELSE ""
    END AS Результат
FROM
    student
    JOIN student_progress USING(student_id)
ORDER BY
    Прогресс DESC,
    Студент ASC;
```

## 3.5.8 Оконные функции, оператор OVER, ORDER BY. Задание

Для студента с именем student_61 вывести все его попытки: название шага, результат и дату отправки попытки (**submission_time**). Информацию отсортировать по дате отправки попытки и указать, сколько минут прошло между отправкой соседних попыток. Название шага ограничить 20 символами и добавить "**...**". Столбцы назвать **Студент, Шаг, Результат, Дата_отправки, Разница**.

```mysql
SELECT
    student_name AS Студент,
    CONCAT(LEFT(CONCAT(step_name), 20), '...') AS Шаг,
    result AS Результат,
    FROM_UNIXTIME(submission_time) AS Дата_отправки,
    SEC_TO_TIME(submission_time - LAG(submission_time, 1, submission_time) OVER (ORDER BY submission_time)) AS Разница
FROM
    student 
    JOIN step_student USING(student_id)
    JOIN step USING(step_id)
WHERE
    student_name = 'student_61'
ORDER BY
    submission_time;
```

## 3.5.9 Задание

Посчитать среднее время, за которое пользователи проходят урок по следующему алгоритму:
* для каждого пользователя вычислить время прохождения **шага** как сумму времени, потраченного на каждую попытку (время попытки - это разница между временем отправки задания и временем начала попытки), при этом попытки, которые длились больше 4 часов не учитывать, так как пользователь мог просто оставить задание открытым в браузере, а вернуться к нему на следующий день;
* для каждого студента посчитать общее время, которое он затратил на каждый урок;
* вычислить среднее время выполнения урока в часах, результат округлить до 2-х знаков после запятой;
* вывести информацию по возрастанию времени, пронумеровав строки, для каждого урока указать номер модуля и его позицию в нем.  

Столбцы результата назвать **Номер, Урок, Среднее_время**.

```mysql
WITH query_in_1 (student_id, module_id, lesson_position, lesson_name, step_time) AS (
    SELECT
        student_id,
        module_id,
        lesson_position,
        lesson_name,
        SUM(submission_time - attempt_time) AS step_time
    FROM
        module
        JOIN lesson USING(module_id)
        JOIN step USING(lesson_id)
        JOIN step_student USING(step_id)
    WHERE
        submission_time - attempt_time < 4 * 3600
    GROUP BY
        student_id, module_id, lesson_position, lesson_name
    )
SELECT
    ROW_NUMBER() OVER (ORDER BY ROUND(AVG(lesson_time), 2) ASC) AS Номер,
    Урок,
    ROUND(AVG(lesson_time), 2) AS Среднее_время
FROM (
    SELECT
        CONCAT(module_id, '.', lesson_position, ' ', lesson_name) AS Урок,
        step_time / 3600 as lesson_time
    FROM
        query_in_1 ) query_in_2
GROUP BY
    Урок;
```

## 3.5.10 Задание

```mysql

```

## 3.5.11 Задание

```mysql

```

## 3.5.12 Задание

```mysql

```

## 3.5.13 Задание

```mysql

```